In [1]:
import os, csv
import pandas as pd

In [260]:
#defining the categories
categoryDefinition = {'398':['398'], 'Function': ['686', '685', '683', '687', '628', '477', '252'],
                      'Memory':['788', '401', '476', '467', '562', '190'],
                      'Evaluation':['571', '595', '570', '783', '768'],
                      'Initialization':['457','665'],
                      'Reachability':['561','758'],
                      'Resource':['664','672'],
                      'Conversion':['704','195'],
                      'Calculation':['369','682']} #key = category, value = corresponding cwe types

#returns 1 or more categories associated with the CWE types in the parameter cwe
def get_categories(cwe):
    res = []
    for k,v in categoryDefinition.items():
        if any(c in v for c in cwe.split(',')):
            res.append(k)
    return res

In [261]:
#for each site, first load the weak snippets, then load the post information, and then merge the two to create a df containing both snippet + year
#at end, merge all three into one df
#we do this to avoid potential conflicts with duplicate post ids among the different sites

#SO
so_dfs = []
so_csvs = ['so-cWeakSnippets.csv','so-cppWeakSnippets.csv']
for c in so_csvs:
    so_dfs.append(pd.read_csv(c))
so_df = pd.concat(so_dfs,ignore_index=True)
so_df[['PostId','extra']] = so_df['SnippetId'].str.split('-',expand=True)
so_df = so_df.drop('extra',axis=1)

postInfo_df = pd.read_csv('Post_Information-so.csv')
postInfo_df = postInfo_df[['PostId','CreationDate']]
postInfo_df['CreationDate'] = postInfo_df['CreationDate'].str[:4]
postInfo_df.rename(columns={'CreationDate': 'Year'}, inplace=True)

postInfo_df['PostId'] = postInfo_df['PostId'].astype(str)
so_df = pd.merge(so_df, postInfo_df, left_on='PostId', right_on='PostId')


#ARD
ard_df = pd.read_csv('ardWeakSnippets.csv')
ard_df[['PostId','extra']] = ard_df['SnippetId'].str.split('-',expand=True)
ard_df = ard_df.drop('extra',axis=1)

postInfo_df_ard = pd.read_csv('Post_Information-ard.csv')

postInfo_df_ard = postInfo_df_ard[['PostId','CreationDate']]
postInfo_df_ard['CreationDate'] = postInfo_df_ard['CreationDate'].str[:4]
postInfo_df_ard.rename(columns={'CreationDate': 'Year'}, inplace=True)

postInfo_df_ard['PostId'] = postInfo_df_ard['PostId'].astype(str)
ard_df = pd.merge(ard_df, postInfo_df_ard, left_on='PostId', right_on='PostId')


#RP
rp_df = pd.read_csv('rpWeakSnippets.csv')
rp_df[['PostId','extra']] = rp_df['SnippetId'].str.split('-',expand=True)
rp_df = rp_df.drop('extra',axis=1)

postInfo_df_rp = pd.read_csv('Post_Information-rp.csv')

postInfo_df_rp = postInfo_df_rp[['PostId','CreationDate']]
postInfo_df_rp['CreationDate'] = postInfo_df_rp['CreationDate'].str[:4]
postInfo_df_rp.rename(columns={'CreationDate': 'Year'}, inplace=True)

postInfo_df_rp['PostId'] = postInfo_df_rp['PostId'].astype(str)
rp_df = pd.merge(rp_df, postInfo_df_rp, left_on='PostId', right_on='PostId')

#concat all
df = pd.concat([so_df,ard_df,rp_df], ignore_index=True)

#get categories and store in new column
df['Category'] = df['CWEs'].apply(get_categories)
print(df)

      SnippetId CWEs Site    PostId  Year          Category
0    10687971-2  758   so  10687971  2012    [Reachability]
1      11069389  398   so  11069389  2012             [398]
2      12483533  595   so  12483533  2012      [Evaluation]
3      12666409  457   so  12666409  2012  [Initialization]
4      14166964  398   so  14166964  2013             [398]
..          ...  ...  ...       ...   ...               ...
600       78348  398   rp     78348  2018             [398]
601       79128  398   rp     79128  2018             [398]
602       81747  398   rp     81747  2018             [398]
603       89185  398   rp     89185  2018             [398]
604       92355  398   rp     92355  2018             [398]

[605 rows x 6 columns]


Number of Weak Code Snippets by Cateogry by Year - #Weak Code snippets(𝐶, 𝑦)

In [262]:
# in a new df, store each unique category in seperate row

df_cat = df.explode('Category').reset_index(drop=True)
df_cat
# get the number of weak code snippets by year for each category
snippetsByCatByYear = df_cat.groupby(['Category','Year']).count()
snippetsByCatByYear

SnippetId  CWEs  Site  PostId
Category     Year                               
398          2009          2     2     2       2
             2010          4     4     4       4
             2011          3     3     3       3
             2012          3     3     3       3
             2013         22    22    22      22
...                      ...   ...   ...     ...
Reachability 2019          1     1     1       1
             2021          1     1     1       1
Resource     2014          1     1     1       1
             2019          1     1     1       1
             2021          1     1     1       1

[72 rows x 4 columns]

Number of Weak Code Snippets by Year - #Weak Code snippets(𝑦)

In [263]:
snippetsByYear = df.groupby('Year').count()
snippetsByYear

,SnippetId,CWEs,Site,PostId,Category
Year,,,,,
2009,2,2,2,2,2
2010,6,6,6,6,6
2011,4,4,4,4,4
2012,6,6,6,6,6
2013,30,30,30,30,30
2014,74,74,74,74,74
2015,78,78,78,78,78
2016,100,100,100,100,100
2017,72,72,72,72,72


Number of Weak Code Snippets by Year - #Weak Code snippets(𝑦) (excluding posts that only contain CWE 398)

In [264]:
#first, need to remove posts that only contain CWE 398, this is done by storing each CWE a snippet has in a seperate row, and then dropping rows that contain 398
dfNo398 = df.astype({"CWEs": str})
dfNo398['CWEs'] = df['CWEs'].str.split(",")
dfNo398 = dfNo398.explode('CWEs').reset_index(drop=True)
dfNo398 = dfNo398.drop(dfNo398[(dfNo398['CWEs']=='398')].index)
snippetsByYearNo398 = dfNo398[['Year','SnippetId']]

#group the non398 snippets by year
snippetsByYearNo398 = snippetsByYearNo398.groupby('Year').count()
snippetsByYearNo398

,SnippetId
Year,
2010,4
2011,1
2012,3
2013,11
2014,38
2015,36
2016,38
2017,20
2018,27


Number of snippets BY SITE per year

In [265]:
snippetsBySiteByYear = df.groupby(['Site','Year']).count()
snippetsBySiteByYear

SnippetId  CWEs  PostId  Category
Site Year                                   
ard  2014         15    15      15        15
     2015         43    43      43        43
     2016         50    50      50        50
     2017         43    43      43        43
     2018         45    45      45        45
     2019         44    44      44        44
     2020         47    47      47        47
     2021          8     8       8         8
rp   2013          5     5       5         5
     2014          9     9       9         9
     2015          3     3       3         3
     2016          5     5       5         5
     2017          3     3       3         3
     2018          5     5       5         5
     2019          1     1       1         1
     2020          1     1       1         1
so   2009          2     2       2         2
     2010          6     6       6         6
     2011          4     4       4         4
     2012          6     6       6         6
     2013         25    25      25        25
     2014         50    50      50        50
     2015         32    32      32        32
     2016         45    45      45        45
     2017         26    26      26        26
     2018         50    50      50        50
     2019         32    32      32        32

Number of snippets BY SITE per year (exluding snippets with only 398)

In [266]:
snippetsBySiteByYearNo398 = dfNo398.groupby(['Site','Year']).count() #exclude snippets that ONLY contain 398
snippetsBySiteByYearNo398

SnippetId  CWEs  PostId  Category
Site Year                                   
ard  2014         13    13      13        13
     2015         13    13      13        13
     2016         19    19      19        19
     2017         12    12      12        12
     2018         16    16      16        16
     2019         19    19      19        19
     2020         23    23      23        23
     2021          8     8       8         8
rp   2013          5     5       5         5
     2014          4     4       4         4
     2015          2     2       2         2
     2016          4     4       4         4
     2017          3     3       3         3
     2019          1     1       1         1
     2020          1     1       1         1
so   2010          4     4       4         4
     2011          1     1       1         1
     2012          3     3       3         3
     2013          6     6       6         6
     2014         21    21      21        21
     2015         21    21      21        21
     2016         15    15      15        15
     2017          5     5       5         5
     2018         11    11      11        11
     2019         10    10      10        10

Total number of IoT posts by Year - #IoT Posts(𝑦)

In [268]:
# get the number of all IoT posts by year 
# first need to concat the post information for all three sites
allPosts = pd.concat([postInfo_df,postInfo_df_ard,postInfo_df_rp],ignore_index=True)
# group by year
IoTPostsByYear = allPosts.groupby('Year').count()
IoTPostsByYear

#store all in one excel book
with pd.ExcelWriter('trendAnalysis.xlsx') as writer:
    snippetsByCatByYear.to_excel(writer, sheet_name='CatByYear')
    snippetsByYear.to_excel(writer, sheet_name='ByYear')
    snippetsByYearNo398.to_excel(writer, sheet_name='ByYearNo398')
    IoTPostsByYear.to_excel(writer, sheet_name='AllPostsByYear')
    snippetsBySiteByYear.to_excel(writer,sheet_name='BySite')
    snippetsBySiteByYearNo398.to_excel(writer,sheet_name='BySiteNo398')